In [ ]:
from pylab import rcParams
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import os 
import sys
import numpy as np
import pandas as pd

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import h5py

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/My\ Drive/ElPhotons/

/content/drive/My Drive/ElPhotons


In [ ]:
# Open the input HDF5 file
dsets = h5py.File('SinglePhotonPt50_IMGCROPS_n249k_RHv2.hdf5', 'r')
keys = ['X','y'] # key names in in put hdf5


In [ ]:
temp=dsets['X'].value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """Entry point for launching an IPython kernel.


In [ ]:
ytemp=dsets['y'].value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """Entry point for launching an IPython kernel.


In [ ]:
import torch
temp2=torch.from_numpy(temp)

In [ ]:
graphs=[]

for i in range(temp2.shape[0]):

  ecal=temp2[i][:,:,0].cuda()

  xhit2,yhit2=torch.nonzero(ecal,as_tuple=True)  ## Select hits in detector

  eneEcal=ecal[xhit2,yhit2] ## Select energies of hits

  feats=torch.stack((xhit2.float(),yhit2.float(),eneEcal),dim=1) ## concatenate x,y locations and energies (3 features in total)

  cords=feats[:,[0,1]] ## cords = x,y coordinates 

  graphs.append(cords)

In [ ]:
lengs=[]
for kk in graphs:
  lengs.append(len(kk))

In [ ]:
lengs=np.array(lengs)

In [ ]:
outers=np.where(lengs==0)[0].tolist()

In [ ]:
inners=np.where(lengs!=0)[0].tolist()

In [ ]:
photons = h5py.File('PhotonData.h5', 'w')

In [ ]:
photons.create_dataset('X', data=temp[inners])

<HDF5 dataset "X": shape (248693, 32, 32, 2), type "<f4">

In [ ]:
photons.create_dataset('y', data=ytemp[inners])

<HDF5 dataset "y": shape (248693,), type "<f4">

In [ ]:
photons.close()

In [ ]:
dsets = h5py.File('SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'w')

In [ ]:
dsets.__delitem__('X2')

In [ ]:
dsets.create_dataset('X2', data=res)

<HDF5 dataset "X2": shape (248693, 32, 32, 2), type "<f4">

In [ ]:

res = np.delete(temp, outers)  # delete element with index 1, i.e. second element
dsets['X2'] = res  # reassign to dataset

In [ ]:
res=temp[inners]

In [ ]:
len(inners)

248693

In [ ]:
res.shape

(248693, 32, 32, 2)

In [ ]:
def np2arrowArray(x):
    if len(x.shape) > 1:
        x = np.transpose(x, [2,0,1])
        return pa.array([x.tolist()])
    else:
        return pa.array([x.tolist()])

    
def convert_to_Parquet(filename, start, stop, chunk_size):
    # Open the input HDF5 file
    dsets = h5py.File(filename, 'r')
    keys = ['X','y'] # key names in in put hdf5
    row0 = [np2arrowArray(dsets[key][0]) for key in keys]
    keys = ['X','y'] # desired key names in output parquet
    table0 = pa.Table.from_arrays(row0, keys) 
    
    # Open the output Parquet file
    filename = '%s.snappy.parquet'%(filename)
    writer = pq.ParquetWriter(filename, table0.schema, compression='snappy')

    # Loop over file chunks of size chunk_size
    nevts = stop - start
    for i in range(int(np.ceil(1.*nevts/chunk_size))):
        
        begin = start + i*chunk_size
        end = begin + chunk_size

        # Load array chunks into memory
        X = dsets['X'][begin:end]
        label=dsets['y'][begin:end]

        # Convert events in the chunk one-by-one
        print (filename)
        print('Doing events: [%d->%d)'%(begin,end))
        for j in range(len(X)):


          # Create a list for each sample
          sample = [np2arrowArray(X[j]),np2arrowArray(label[j])]

          table = pa.Table.from_arrays(sample, keys)
          os.chdir('\\'.join(os.getcwd().split('\\')[:-1]) + './')
          writer.write_table(table)

    writer.close()
    return filename


In [ ]:
def main():
    wdir = r'./'
    hdf5_filenames = ['PhotonData.h5']
    chunk_size = 2000
    start = 0 
    stop = 248000
    for filename in hdf5_filenames: 
        convert_to_Parquet(filename, start, stop, chunk_size)

In [ ]:
import numpy as np
import h5py

In [ ]:
if __name__ == "__main__":
    main()

PhotonData.h5.snappy.parquet
Doing events: [0->2000)
PhotonData.h5.snappy.parquet
Doing events: [2000->4000)
PhotonData.h5.snappy.parquet
Doing events: [4000->6000)
PhotonData.h5.snappy.parquet
Doing events: [6000->8000)
PhotonData.h5.snappy.parquet
Doing events: [8000->10000)
PhotonData.h5.snappy.parquet
Doing events: [10000->12000)
PhotonData.h5.snappy.parquet
Doing events: [12000->14000)
PhotonData.h5.snappy.parquet
Doing events: [14000->16000)
PhotonData.h5.snappy.parquet
Doing events: [16000->18000)
PhotonData.h5.snappy.parquet
Doing events: [18000->20000)
PhotonData.h5.snappy.parquet
Doing events: [20000->22000)
PhotonData.h5.snappy.parquet
Doing events: [22000->24000)
PhotonData.h5.snappy.parquet
Doing events: [24000->26000)
PhotonData.h5.snappy.parquet
Doing events: [26000->28000)
PhotonData.h5.snappy.parquet
Doing events: [28000->30000)
PhotonData.h5.snappy.parquet
Doing events: [30000->32000)
PhotonData.h5.snappy.parquet
Doing events: [32000->34000)
PhotonData.h5.snappy.parqu

In [ ]:
1+1